# Pulmonary Fibrosis Progression Prediction

Created by: Arnav Malhotra

This is to build the model for the 2020 Kaggle Competition entitled OSIC Pulmonary Fibrosis Progression where the goal is to predict lung function decline.
It can be found at https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/overview

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

In [3]:
# define where the data is located
path = Path('osic-pulmonary-fibrosis-progression')
path

WindowsPath('osic-pulmonary-fibrosis-progression')

In [4]:
df_train = pd.read_csv(path/'train.csv')
df_train.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [6]:
df_test = pd.read_csv(path/'test.csv')
df_test.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264,6,3020,70.186855,73,Male,Ex-smoker
1,ID00421637202311550012437,15,2739,82.045291,68,Male,Ex-smoker
2,ID00422637202311677017371,6,1930,76.672493,73,Male,Ex-smoker
3,ID00423637202312137826377,17,3294,79.258903,72,Male,Ex-smoker
4,ID00426637202313170790466,0,2925,71.824968,73,Male,Never smoked
